In [9]:
import math
import random
import numpy as np
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [144]:
class Value:
    
    def __init__(self, data, _children=(), _op='', label = ''):
        self.data = data #current value we are at
        self.grad = 0 #effect of gradient on final variable
        self._backward = lambda: None #function
        self._prev = set(_children) #variables in previous operation
        self._op = _op #operation on the children
        self.label = label #variable label
        
    def __repr__(self):
        return f"{self.label} -> Value(data = {self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        output = Value(self.data + other.data, (self, other), '+')
        def _backward(): 
            self.grad += output.grad
            other.grad += output.grad
        output._backward = _backward
        return output
    
    def __radd__(self, other): # other + self
        return self + other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        output = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data*output.grad
            other.grad += self.data*output.grad
        output._backward = _backward
        return output
    
    def __rmul__(self, other):
        return self * other
    
    def __neg__(self):
        return self*-1
    
    def __sub__(self, other):
        return self + (-other)
    
    def __pow__(self, other):
        assert isinstance(other, (int,float))
        output = Value(self.data**other, (self,), f'^{other}')
        def _backward():
            self.grad += other*(self.data**(other - 1))*output.grad
        output._backward = _backward
        return output
    
    def __truediv__(self, other):
        return self * (other**-1)
    
    def exp(self):
        output_data = math.exp(self.data)
        output = Value(output_data, (self, ),'exp')
        def _backward():
            self.grad += output_data*output.grad
        output._backward = _backward
        return output
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) +  1)
        output = Value(t, (self, ),'tanh')
        def _backward():
            self.grad += (1 - t**2)*output.grad
        output._backward = _backward
        return output
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1
        for node in reversed(topo):
            node._backward()

In [4]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(-10.0, label = 'c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d*f; L.label = 'L'; L.grad = 1.0

In [6]:
L**2

 -> Value(data = 1024.0)

In [7]:
#using pytorch
import torch

x1 = torch.Tensor([2.0]).double()
x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()
x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()
w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()
w2.requires_grad = True
b = torch.Tensor([6.88]).double()
b.requires_grad =  True
    
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

o.backward()

print(o.data.item())
print(x1.grad.item())

0.7064193777288968
-1.5029149883073543


In [186]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        return act.tanh()
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs 
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP: #multilayer perceptron
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for neuron in self.layers for p in neuron.parameters()]
        
n = MLP(3, [4,4,1])
n(x)

 -> Value(data = -0.9453981273131602)

In [220]:
#implementing basic learning

#forward pass to get prediction value
xs = [[2, 3, -1], [3, -1, 0.5], [0.5, 1, 1], [1, 1, -1]]
ys = [1, -1, -1, 1]
ypred = [n(x) for x in xs]

ypred

[ -> Value(data = 0.9901685376766685),
  -> Value(data = -0.995509402940971),
  -> Value(data = -0.996670572455904),
  -> Value(data = 0.9904163596090751)]

In [219]:
#calculate error
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
print(loss)
#need to zero grad before backpropagating
for p in n.parameters():
    p.grad = 0
loss.backward()

 -> Value(data = 0.008480178423822121)


In [218]:
#backward pass where we adjust based on gradient
for p in n.parameters():
    p.data -= 0.1*p.grad